In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
train = pd.read_csv('/gdrive/My Drive/Data/trains.csv')
test = pd.read_csv('/gdrive/My Drive/Data/test.csv')

for col in train.columns:
    col_type = train[col].dtypes
    min1 = train[col].min()
    max1 = train[col].max()
    if str(col_type)[:3] == 'int':
        train[col] = train[col].astype(np.int16)
    else:
        if min1 > np.finfo(np.float16).min and max1 < np.finfo(np.float16).max:
            train[col] = train[col].astype(np.float16)
        elif min1 > np.finfo(np.float32).min and max1 < np.finfo(np.float32).max:
            train[col] = train[col].astype(np.float32)
        else:
            train[col] = train[col].astype(np.float64)
train = train.sample(frac=1,random_state=123).reset_index(drop=True)
train_X = train.iloc[:,4:]
train_Y = train.iloc[:,0:4]
test_X = test.iloc[:,1:]

In [3]:
from keras import backend as K

def mish(x):
        return x * K.tanh( K.softplus(x))

#K.get_custom_objects().update({'mish': Activation(mish)})

Using TensorFlow backend.


In [4]:
train_set = train_X.iloc[:690000,:], train_Y.iloc[:690000,:]
val_set = train_X.iloc[690000:,:], train_Y.iloc[690000:,:]
print(train_set[0].shape)
print(val_set[0].shape)

(690000, 226)
(120000, 226)


In [0]:
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras import optimizers
from keras import activations


def build_model(input_s,dropout_rate1,lr):
    model = Sequential()
    model.add(Dense(units=814, activation=mish, input_dim=input_s))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate1))
    model.add(Dense(units=724, activation=mish, input_dim=226))
    model.add(Dense(units=634, activation=mish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate1))
    model.add(Dense(units=544, activation=mish, input_dim=226))
    model.add(Dense(units=454, activation=mish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate1))
    model.add(Dense(units=364, activation=mish, input_dim=226))
    model.add(Dense(units=274, activation=mish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate1))
    model.add(Dense(units=184, activation=mish, input_dim=226))
    model.add(Dense(units=94, activation=mish, input_dim=226))
    model.add(Dense(units=4, activation='linear'))
    op = optimizers.Nadam(lr = lr , beta_1 = 0.9, beta_2 = 0.999)
    model.compile(loss='mae', optimizer=op, metrics=['mae'])
    return model

In [0]:
def fit_with(verbose, lr, dropout_rate1):
    model = build_model(226, dropout_rate1, lr)
    fit_with = model.fit(train_set[0],train_set[1], epochs=8, batch_size=1000, shuffle=True)
    e_val = model.evaluate(val_set[0],val_set[1],steps=10, verbose=0)
    print('Test loss:', e_val[0])
    print('Test score:', e_val[1])
    return -(e_val[0])

from functools import partial

verbose = 1
fit_with_partial = partial(fit_with, verbose)

In [0]:
new_model = keras.models.load_model('/gdrive/My Drive/ckpt/tuning_254_valloss1.90.hdf5',custom_objects={'mish': mish})
new_model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_221 (Dense)            (None, 239)               54253     
_________________________________________________________________
batch_normalization_89 (Batc (None, 239)               956       
_________________________________________________________________
dropout_89 (Dropout)         (None, 239)               0         
_________________________________________________________________
dense_222 (Dense)            (None, 252)               60480     
_________________________________________________________________
dense_223 (Dense)            (None, 252)               63756     
_________________________________________________________________
batch_normalization_90 (Batc (None, 252)               10

In [7]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=c73182cc904e2c24935ba78dc871a24b2f76d4f93d599f8cb0b34e57845123dd
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [15]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'dropout_rate1': (0.0, 0.3), 
           'lr': (1e-4, 1e-2)}

bayes_optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

bayes_optimizer.maximize(init_points=15, n_iter=15)
for i, res in enumerate(bayes_optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(bayes_optimizer.max)

|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------
Epoch 1/8
690000/690000 [==============================] - 20s 30us/step - loss: 54.0470 - mean_absolute_error: 54.0470
Epoch 2/8
690000/690000 [==============================] - 16s 23us/step - loss: 23.9981 - mean_absolute_error: 23.9981
Epoch 3/8
690000/690000 [==============================] - 16s 23us/step - loss: 14.5659 - mean_absolute_error: 14.5659
Epoch 4/8
690000/690000 [==============================] - 16s 23us/step - loss: 11.8464 - mean_absolute_error: 11.8464
Epoch 5/8
690000/690000 [==============================] - 16s 22us/step - loss: 10.4215 - mean_absolute_error: 10.4215
Epoch 6/8
690000/690000 [==============================] - 16s 23us/step - loss: 9.5170 - mean_absolute_error: 9.5170
Epoch 7/8
690000/690000 [==============================] - 15s 22us/step - loss: 8.8122 - mean_absolute_error: 8.8122
Epoch 8/8
690000/690000 [==============================] - 16s 2

KeyboardInterrupt: ignored

In [16]:
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import keras.backend as K
plt.figure(figsize = (13,8))
class LRFinder(Callback):
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None):
        super().__init__()
        
        self.min_lr = min_lr
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')
        plt.ylim(0, 10)

early_stop = keras.callbacks.EarlyStopping(patience=50, monitor='val_loss')

ckpt_dir = '/gdrive/My Drive/ckpt1'
ckpt_path = ckpt_dir + '/tuning_{epoch:02d}_valloss{val_loss:.2f}.hdf5'
ckpt = keras.callbacks.ModelCheckpoint(ckpt_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')


model = build_model(226, dropout_rate1= 0.02551, lr = 0.000486)
history = model.fit(train_X,train_Y, epochs=300, callbacks = [early_stop, ckpt] ,validation_split = 0.1, batch_size=1000, shuffle=True)

Train on 729000 samples, validate on 81000 samples
Epoch 1/300
729000/729000 [==============================] - 34s 46us/step - loss: 59.8009 - mean_absolute_error: 59.8009 - val_loss: 47.5886 - val_mean_absolute_error: 47.5886
Epoch 2/300
729000/729000 [==============================] - 18s 24us/step - loss: 23.0263 - mean_absolute_error: 23.0263 - val_loss: 14.2240 - val_mean_absolute_error: 14.2240
Epoch 3/300
729000/729000 [==============================] - 18s 25us/step - loss: 11.4416 - mean_absolute_error: 11.4416 - val_loss: 8.6951 - val_mean_absolute_error: 8.6951
Epoch 4/300
729000/729000 [==============================] - 18s 25us/step - loss: 8.7425 - mean_absolute_error: 8.7425 - val_loss: 6.3866 - val_mean_absolute_error: 6.3866
Epoch 5/300
729000/729000 [==============================] - 18s 24us/step - loss: 7.4609 - mean_absolute_error: 7.4609 - val_loss: 5.3748 - val_mean_absolute_error: 5.3748
Epoch 6/300
729000/729000 [==============================] - 18s 25us/step

KeyboardInterrupt: ignored

<Figure size 936x576 with 0 Axes>

In [7]:
model = build_model(226, dropout_rate1= 0.02551, lr = 0.000486)
ckpt_dir = '/gdrive/My Drive/ckpt1'
model.load_weights(ckpt_dir +'/tuning_270_valloss0.84.hdf5')
y1 = model.predict(train_X)
model.load_weights(ckpt_dir +'/tuning_227_valloss0.95.hdf5')
y2 = model.predict(train_X)
model.load_weights(ckpt_dir +'/tuning_195_valloss0.98.hdf5')
y3 = model.predict(train_X)
model.load_weights(ckpt_dir +'/tuning_221_valloss0.96.hdf5')
y4 = model.predict(train_X)
model.load_weights(ckpt_dir +'/tuning_259_valloss0.89.hdf5')
y5 = model.predict(train_X)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
model.load_weights(ckpt_dir +'/tuning_270_valloss0.84.hdf5')
pred_y1 = model.predict(test_X)
model.load_weights(ckpt_dir +'/tuning_227_valloss0.95.hdf5')
pred_y2 = model.predict(test_X)
model.load_weights(ckpt_dir +'/tuning_195_valloss0.98.hdf5')
pred_y3 = model.predict(test_X)
model.load_weights(ckpt_dir +'/tuning_221_valloss0.96.hdf5')
pred_y4 = model.predict(test_X)
model.load_weights(ckpt_dir +'/tuning_259_valloss0.89.hdf5')
pred_y5 = model.predict(test_X)

In [9]:
frame1, frame2, frame3, frame4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
frame1["model1"] = y1[:,0]
frame1["model2"] = y2[:,0]
frame1["model3"] = y3[:,0]
frame1["model4"] = y4[:,0]
frame1["model5"] = y5[:,0]

frame2["model1"] = y1[:,1]
frame2["model2"] = y2[:,1]
frame2["model3"] = y3[:,1]
frame2["model4"] = y4[:,1]
frame2["model5"] = y5[:,1]

frame3["model1"] = y1[:,2]
frame3["model2"] = y2[:,2]
frame3["model3"] = y3[:,2]
frame3["model4"] = y4[:,2]
frame3["model5"] = y5[:,2]

frame4["model1"] = y1[:,3]
frame4["model2"] = y2[:,3]
frame4["model3"] = y3[:,3]
frame4["model4"] = y4[:,3]
frame4["model5"] = y5[:,3]
!pip install lightgbm

In [14]:
from sklearn.metrics import mean_absolute_error

mean1 = np.mean(frame3,axis=1)
mean_absolute_error(train_Y["layer_3"], mean1)

0.57774216

In [0]:
fr1, fr2, fr3, fr4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
fr1["model1"] = pred_y1[:,0]
fr1["model2"] = pred_y2[:,0]
fr1["model3"] = pred_y3[:,0]
fr1["model4"] = pred_y4[:,0]
fr1["model5"] = pred_y5[:,0]

fr2["model1"] = pred_y1[:,1]
fr2["model2"] = pred_y2[:,1]
fr2["model3"] = pred_y3[:,1]
fr2["model4"] = pred_y4[:,1]
fr2["model5"] = pred_y5[:,1]

fr3["model1"] = pred_y1[:,2]
fr3["model2"] = pred_y2[:,2]
fr3["model3"] = pred_y3[:,2]
fr3["model4"] = pred_y4[:,2]
fr3["model5"] = pred_y5[:,2]

fr4["model1"] = pred_y1[:,3]
fr4["model2"] = pred_y2[:,3]
fr4["model3"] = pred_y3[:,3]
fr4["model4"] = pred_y4[:,3]
fr4["model5"] = pred_y5[:,3]



In [21]:
import lightgbm as lgb
lgb_param = {'boosting_type':'gbdt',
             'num_leaves': 50,
             'max_depth': -1,
            'learning_rate': 0.005, 
            'bagging_fraction' : 1,
            'max_bin' : 5000 ,
            'bagging_freq': 20,
            'colsample_bytree': 0.6,
             'metric': 'mae',
             'min_split_gain': 0.5,
            'min_child_weight': 1,
            'min_child_samples': 20,
            'scale_pos_weight':1,
            'zero_as_missing': True,
            'objective': 'regression', # 목적 함수
            }
# train_set = lgb.Dataset(frame1, train_Y["layer_1"], silent=False)
train_set = lgb.Dataset(frame1.iloc[:690000,:], train_Y.iloc[:690000,:]["layer_1"], silent=False)
valid_set = lgb.Dataset(frame1.iloc[690000:,:], train_Y.iloc[690000:,:]["layer_1"], silent=False)
lgb_model = lgb.train(params = lgb_param, train_set = train_set , num_boost_round= 3000 ,verbose_eval=50 , valid_sets=valid_set)
predict1 = lgb_model.predict(fr1)

train_set = lgb.Dataset(frame2.iloc[:690000,:], train_Y.iloc[:690000,:]["layer_2"], silent=False)
valid_set = lgb.Dataset(frame2.iloc[690000:,:], train_Y.iloc[690000:,:]["layer_2"], silent=False)
lgb_model = lgb.train(params = lgb_param, train_set = train_set , num_boost_round= 3000 ,verbose_eval=50, valid_sets=valid_set)
predict2 = lgb_model.predict(fr2)

train_set = lgb.Dataset(frame3.iloc[:690000,:], train_Y.iloc[:690000,:]["layer_3"], silent=False)
valid_set = lgb.Dataset(frame3.iloc[690000:,:], train_Y.iloc[690000:,:]["layer_3"], silent=False)
lgb_model = lgb.train(params = lgb_param, train_set = train_set , num_boost_round= 3000 ,verbose_eval=50, valid_sets=valid_set)
predict3 = lgb_model.predict(fr3)

train_set = lgb.Dataset(frame4.iloc[:690000,:], train_Y.iloc[:690000,:]["layer_4"], silent=False)
valid_set = lgb.Dataset(frame4.iloc[690000:,:], train_Y.iloc[690000:,:]["layer_4"], silent=False)
lgb_model = lgb.train(params = lgb_param, train_set = train_set , num_boost_round= 3000 ,verbose_eval=50, valid_sets=valid_set)
predict4 = lgb_model.predict(fr4)

[50]	valid_0's l1: 58.4758
[100]	valid_0's l1: 45.5122
[150]	valid_0's l1: 35.4225
[200]	valid_0's l1: 27.5698
[250]	valid_0's l1: 21.4581
[300]	valid_0's l1: 16.7013
[350]	valid_0's l1: 12.9993
[400]	valid_0's l1: 10.1183
[450]	valid_0's l1: 7.87617
[500]	valid_0's l1: 6.13136
[550]	valid_0's l1: 4.77385
[600]	valid_0's l1: 3.71789
[650]	valid_0's l1: 2.89643
[700]	valid_0's l1: 2.25756
[750]	valid_0's l1: 1.76056
[800]	valid_0's l1: 1.37392
[850]	valid_0's l1: 1.07318
[900]	valid_0's l1: 0.839279
[950]	valid_0's l1: 0.657332
[1000]	valid_0's l1: 0.515862
[1050]	valid_0's l1: 0.40583
[1100]	valid_0's l1: 0.320268
[1150]	valid_0's l1: 0.253777
[1200]	valid_0's l1: 0.202087
[1250]	valid_0's l1: 0.161929
[1300]	valid_0's l1: 0.130728
[1350]	valid_0's l1: 0.106457
[1400]	valid_0's l1: 0.0876138
[1450]	valid_0's l1: 0.0729437
[1500]	valid_0's l1: 0.0615477
[1550]	valid_0's l1: 0.0526771
[1600]	valid_0's l1: 0.0457931
[1650]	valid_0's l1: 0.0404386
[1700]	valid_0's l1: 0.036324
[1750]	valid

In [0]:
sample = pd.read_csv('/gdrive/My Drive/Data/sample_submission.csv')
sample.iloc[:,1] = predict1
sample.iloc[:,2] = predict2
sample.iloc[:,3] = predict3
sample.iloc[:,4] = predict4

In [0]:
fr1, fr2, fr3, fr4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
fr1["model1"] = pred_y1[:,0]
fr1["model2"] = pred_y2[:,0]
fr1["model3"] = pred_y3[:,0]
fr1["model4"] = pred_y4[:,0]
fr1["model5"] = pred_y5[:,0]

fr2["model1"] = pred_y1[:,1]
fr2["model2"] = pred_y2[:,1]
fr2["model3"] = pred_y3[:,1]
fr2["model4"] = pred_y4[:,1]
fr2["model5"] = pred_y5[:,1]

fr3["model1"] = pred_y1[:,2]
fr3["model2"] = pred_y2[:,2]
fr3["model3"] = pred_y3[:,2]
fr3["model4"] = pred_y4[:,2]
fr3["model5"] = pred_y5[:,2]

fr4["model1"] = pred_y1[:,3]
fr4["model2"] = pred_y2[:,3]
fr4["model3"] = pred_y3[:,3]
fr4["model4"] = pred_y4[:,3]
fr4["model5"] = pred_y5[:,3]

sample = pd.read_csv('/gdrive/My Drive/Data/sample_submission.csv')
sample.iloc[:,1] = np.mean(fr1,axis=1)
sample.iloc[:,2] = np.mean(fr2,axis=1)
sample.iloc[:,3] = np.mean(fr3,axis=1)
sample.iloc[:,4] = np.mean(fr4,axis=1)

In [0]:
sample = pd.read_csv('/gdrive/My Drive/Data/sample_submission.csv')
sample.iloc[:,1:] = pred_y
sample.head()

,id,layer_1,layer_2,layer_3,layer_4
0,0,253.438278,228.213654,130.537521,87.765846
1,1,159.621536,126.801979,236.580307,99.874466
2,2,150.084274,182.445572,269.389252,159.521561
3,3,90.494621,227.391785,190.062576,80.387878
4,4,270.253876,292.293335,247.717560,270.892792


In [0]:
sample.to_csv('/gdrive/My Drive/Data/sample6.csv',index=False)

In [24]:
sample.head()

,id,layer_1,layer_2,layer_3,layer_4
0,0,250.744475,230.002684,129.999327,88.161403
1,1,159.999870,129.996983,238.136129,100.000353
2,2,150.010535,179.999312,270.659139,158.969473
3,3,90.007678,230.002684,190.000745,80.295131
4,4,269.934769,291.660620,247.865833,270.063364


In [28]:
model = build_model(226, dropout_rate1= 0.02551, lr = 0.000486)
ckpt_path = ckpt_dir + '/tuning_{epoch:02d}_valloss{val_loss:.2f}.hdf5'
ckpt = keras.callbacks.ModelCheckpoint(ckpt_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
model.load_weights(ckpt_dir +'/tuning_270_valloss0.84.hdf5')
hist = model.fit(train_X, train_Y, epochs=50, callbacks = [ckpt] ,validation_split = 0.2, batch_size=1500, shuffle=True)



Train on 648000 samples, validate on 162000 samples
Epoch 1/100
648000/648000 [==============================] - 12s 18us/step - loss: 1.5507 - mean_absolute_error: 1.5507 - val_loss: 1.1054 - val_mean_absolute_error: 1.1054
Epoch 2/100
648000/648000 [==============================] - 10s 16us/step - loss: 1.4873 - mean_absolute_error: 1.4873 - val_loss: 1.3155 - val_mean_absolute_error: 1.3155
Epoch 3/100
648000/648000 [==============================] - 10s 16us/step - loss: 1.4577 - mean_absolute_error: 1.4577 - val_loss: 1.2036 - val_mean_absolute_error: 1.2036
Epoch 4/100
648000/648000 [==============================] - 10s 16us/step - loss: 1.4212 - mean_absolute_error: 1.4212 - val_loss: 0.8356 - val_mean_absolute_error: 0.8356
Epoch 5/100
648000/648000 [==============================] - 10s 15us/step - loss: 1.3999 - mean_absolute_error: 1.3999 - val_loss: 0.8932 - val_mean_absolute_error: 0.8932
Epoch 6/100
648000/648000 [==============================] - 10s 16us/step - loss

KeyboardInterrupt: ignored

In [35]:
ckpt_path = ckpt_dir + '/tuning_2_{epoch:02d}_valloss{val_loss:.2f}.hdf5'
ckpt = keras.callbacks.ModelCheckpoint(ckpt_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
model.load_weights(ckpt_dir +'/tuning_20_valloss0.71.hdf5')
hist = model.fit(train_X, train_Y, epochs=10, callbacks = [ckpt] ,validation_split = 0.2, batch_size=3000, shuffle=True)

Train on 648000 samples, validate on 162000 samples
Epoch 1/50
648000/648000 [==============================] - 7s 12us/step - loss: 1.1437 - mean_absolute_error: 1.1437 - val_loss: 0.7618 - val_mean_absolute_error: 0.7618
Epoch 2/50
648000/648000 [==============================] - 8s 12us/step - loss: 1.1323 - mean_absolute_error: 1.1323 - val_loss: 0.6541 - val_mean_absolute_error: 0.6541
Epoch 3/50
648000/648000 [==============================] - 8s 12us/step - loss: 1.1330 - mean_absolute_error: 1.1330 - val_loss: 0.6494 - val_mean_absolute_error: 0.6494
Epoch 4/50
648000/648000 [==============================] - 8s 12us/step - loss: 1.1349 - mean_absolute_error: 1.1349 - val_loss: 0.6845 - val_mean_absolute_error: 0.6845
Epoch 5/50
648000/648000 [==============================] - 8s 12us/step - loss: 1.1356 - mean_absolute_error: 1.1356 - val_loss: 0.7196 - val_mean_absolute_error: 0.7196
Epoch 6/50
648000/648000 [==============================] - 7s 12us/step - loss: 1.1312 - mea

KeyboardInterrupt: ignored

In [36]:
model.load_weights(ckpt_dir +'/tuning_1_03_valloss0.59.hdf5')
evalu1 = model.predict(test_X)
sample = pd.read_csv('/gdrive/My Drive/Data/sample_submission.csv')
sample.iloc[:,1:] = evalu1
sample.head()

,id,layer_1,layer_2,layer_3,layer_4
0,0,254.607025,230.707794,130.663422,86.190086
1,1,158.474579,127.409348,237.045227,100.103813
2,2,148.316925,178.831528,273.054901,156.929642
3,3,91.614883,229.074127,188.860092,81.682976
4,4,274.956879,295.221405,244.777359,272.859344


In [0]:
sample.to_csv('/gdrive/My Drive/Data/sample7.csv',index=False)